<a href="https://colab.research.google.com/github/NScreator19/cemist-1/blob/main/Cemist7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd

st.title("CEMIST - Cement Emission Simulation and Tracking")

# --- Data Awal
data = {
    "Cement type": [
        "NAR1 PCC (Transisi)", "NAR1 OPC", "NAR1 OPC - SG", "NAR1 RFP", "NAR1 OWC", "NAR1 Type V", "NAR1 PwrPro",
        "NAR2 PCC Regular", "NAR2 RFP", "NAR2 PwrPro",
        "CIL PCC",
        "TQ1 PCC (Transisi)", "TQ1 OPC - SG", "TQ1 RFP", "TQ1 Type V",
        "TQ2 PCC Regular", "TQ2 Type V", "TQ2 RFP", "TQ2 OPC", "TQ2 PCC (Transisi)",
        "LHO PCC Regular", "LHO OPC"
    ],
    "Clinker Consumption (ton)": [
        82808, 192961, 571231, 1050785, 48943, 20070, 124720,
        1469132, 45949, 0,
        1440053,
        79722, 0, 226564, 363551,
        487608, 0, 226564, 305894, 0,
        640132, 283590
    ],
    "Cement Production (ton)": [
        93568, 203975, 649126, 1203648, 50770, 21581, 178172,
        2532986, 50428, 0,
        2482850,
        94029, 0, 260539, 380682,
        777580, 0, 260539, 338676, 0,
        1094242, 331684
    ]
}

# --- Inisialisasi DataFrame
df = pd.DataFrame(data)
df["Clinker Factor (%)"] = (df["Clinker Consumption (ton)"] / df["Cement Production (ton)"]) * 100
df = df.fillna(0)

# --- Hitung Total Awal
total_clinker_awal = df["Clinker Consumption (ton)"].sum()
total_cement = df["Cement Production (ton)"].sum()
cf_awal = total_clinker_awal / total_cement * 100

st.write(f"**Clinker Factor Konsolidasi Saat Ini: {cf_awal:.2f}%**")

# --- Input User
cf_target = st.number_input("🎯 Masukkan target Clinker Factor Konsolidasi (%):", min_value=0.0, max_value=100.0, value=68.00)

semen_dipilih = st.multiselect(
    "🛠️ Pilih tipe semen yang akan disesuaikan konsumsi klinkernya:",
    df["Cement type"].tolist()
)

# --- Proses Kalkulasi jika ada input
if cf_target and semen_dipilih:
    # Target total clinker yang diinginkan
    clinker_total_target = (cf_target / 100) * total_cement
    gap = clinker_total_target - total_clinker_awal

    # Ambil hanya semen yang dipilih
    df_selected = df[df["Cement type"].isin(semen_dipilih)].copy()
    total_cement_selected = df_selected["Cement Production (ton)"].sum()

    # Distribusi gap secara proporsional
    df_selected["Clinker Adjustment"] = df_selected["Cement Production (ton)"] / total_cement_selected * gap
    df_selected["Clinker Baru"] = df_selected["Clinker Consumption (ton)"] + df_selected["Clinker Adjustment"]
    df_selected["CF Baru (%)"] = df_selected["Clinker Baru"] / df_selected["Cement Production (ton)"] * 100

    # --- Simpan Data Sebelum & Sesudah untuk Display
    df_sebelum = df[df["Cement type"].isin(semen_dipilih)][[
        "Cement type", "Cement Production (ton)", "Clinker Consumption (ton)", "Clinker Factor (%)"
    ]].copy()

    df_sesudah = df_selected[[
        "Cement type", "Cement Production (ton)", "Clinker Baru", "CF Baru (%)"
    ]].copy()
    df_sesudah.rename(columns={
        "Clinker Baru": "Clinker Consumption (ton)",
        "CF Baru (%)": "Clinker Factor (%)"
    }, inplace=True)

    # Update df_result untuk semua semen
    df_result = df.copy()
    for idx, row in df_selected.iterrows():
        df_result.loc[df_result["Cement type"] == row["Cement type"], "Clinker Consumption (ton)"] = row["Clinker Baru"]
        df_result.loc[df_result["Cement type"] == row["Cement type"], "Clinker Factor (%)"] = row["CF Baru (%)"]

    cf_final = df_result["Clinker Consumption (ton)"].sum() / total_cement * 100

    # --- Output Tabel Sebelum
    st.subheader("📋 Kondisi Sebelum (Semen yang Dipilih)")
    st.dataframe(df_sebelum.style.format({
        "Clinker Factor (%)": "{:.2f}",
        "Clinker Consumption (ton)": "{:,.0f}"
    }))

    # --- Output Tabel Sesudah
    st.subheader("📊 Kondisi Setelah Penyesuaian (Semen yang Dipilih)")
    st.dataframe(df_sesudah.style.format({
        "Clinker Factor (%)": "{:.2f}",
        "Clinker Consumption (ton)": "{:,.0f}"
    }))

    # --- Output Total Final
    st.subheader("📈 Ringkasan Akhir (Semua Semen)")
    st.write(f"**Clinker Factor Konsolidasi Baru: {cf_final:.2f}%**")
    st.dataframe(df_result.style.format({
        "Clinker Factor (%)": "{:.2f}",
        "Clinker Consumption (ton)": "{:,.0f}"
    }))


ModuleNotFoundError: No module named 'streamlit'